Simple web scrapping example

--------------------------------- STEPS -------------------------------------------------

* Libraries an packages bs4 
* Set URL
* Make it soup
* Find in the html code the data tags you want to find, in this case table with a class
* with the desired table get the titles
* set the pandas df with the column names
* get the data
* put the data into the dataframe
-----------------------------------------------------------------------------------------



In [48]:
from bs4 import BeautifulSoup
import requests

In [49]:
url = 'https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')
#
# print(soup)

In [50]:
tableA = soup.find_all('table')[0]

In [51]:
tableB = soup.find('table', class_= 'wikitable sortable')

In [52]:
world_titles = tableB.find_all('th')

In [53]:
#Notice the use of strip in the inline for to get rid of the \n
world_table_titles = [title.text.strip() for title in world_titles]
world_table_titles = world_table_titles
print(world_table_titles)

['Rank', 'Name', 'Industry', 'Revenue (USD millions)', 'Revenue growth', 'Employees', 'Headquarters']


In [54]:
import pandas as pd

In [55]:
df = pd.DataFrame(columns = world_table_titles)
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters


In [56]:
column_data = tableB.find_all('tr')

In [57]:
for row in column_data[1:]: # iterate all tr
    row_data = row.find_all('td') # get all td's in tr
    individual_row_data = [data.text.strip() for data in row_data] # put all instances of td in a list of the tr
    
    length = len(df) # get the current length of the dataframe
    # using loc you can access the datarow in the dataframe where you want to put your data
    # and then for the next iteration it will be added on. Neat!
    df.loc[length] = individual_row_data 


In [58]:
df

,Rank,Name,Industry,Revenue (USD millions),Revenue growth,Employees,Headquarters
0,1,Walmart,Retail,"611,289",6.7%,"2,100,000","Bentonville, Arkansas"
1,2,Amazon,Retail and cloud computing,"513,983",9.4%,"1,540,000","Seattle, Washington"
2,3,ExxonMobil,Petroleum industry,"413,680",44.8%,"62,000","Spring, Texas"
3,4,Apple,Electronics industry,"394,328",7.8%,"164,000","Cupertino, California"
4,5,UnitedHealth Group,Healthcare,"324,162",12.7%,"400,000","Minnetonka, Minnesota"
...,...,...,...,...,...,...,...
95,96,Best Buy,Retail,"46,298",10.6%,"71,100","Richfield, Minnesota"
96,97,Bristol-Myers Squibb,Pharmaceutical industry,"46,159",0.5%,"34,300","New York City, New York"
97,98,United Airlines,Airline,"44,955",82.5%,"92,795","Chicago, Illinois"
98,99,Thermo Fisher Scientific,Laboratory instruments,"44,915",14.5%,"130,000","Waltham, Massachusetts"


In [61]:
#create output directory if not existent
import os
outname = 'Companies.csv'
outdir = '.output_data/'

''' ENABLE IF YOU WANT TO SAVE THE FILE
if not os.path.exists(outdir):
    os.mkdir(outdir)
full_name = os.path.join(outdir, outname)
'''

full_name = outdir + outname

df.to_csv(full_name, index = False) # index = False so it doesn't export the index for every row

Let's make it more interesting, now I want to know if there's a corelation between number of employees and revenue

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
import re

# Function to clean the data
def clean_data(value):
    # Remove commas
    value = value.replace(',', '')
    # Remove square brackets and anything inside them
    value = re.sub(r'\[.*\]', '', value)
    return value

#change literal numbers to int
# Apply the function to the columns
df['Revenue (USD millions)'] = df['Revenue (USD millions)'].astype(str).apply(clean_data).astype(float).astype(int)
df['Employees'] = df['Employees'].astype(str).apply(clean_data).astype(float).astype(int)

print(df['Revenue (USD millions)'])
print(df['Employees'])

0     611289
1     513983
2     413680
3     394328
4     324162
       ...  
95     46298
96     46159
97     44955
98     44915
99     44200
Name: Revenue (USD millions), Length: 100, dtype: int32
0     2100000
1     1540000
2       62000
3      164000
4      400000
       ...   
95      71100
96      34300
97      92795
98     130000
99      51000
Name: Employees, Length: 100, dtype: int32


In [64]:
# separate target from data


X = pd.to_numeric( df['Employees'], errors='coerce' ) #independent variable
y = pd.to_numeric( df['Revenue (USD millions)'], errors='coerce' ) #dependent variable

print(X)
print(y)


0     2100000
1     1540000
2       62000
3      164000
4      400000
       ...   
95      71100
96      34300
97      92795
98     130000
99      51000
Name: Employees, Length: 100, dtype: int32
0     611289
1     513983
2     413680
3     394328
4     324162
       ...  
95     46298
96     46159
97     44955
98     44915
99     44200
Name: Revenue (USD millions), Length: 100, dtype: int32


In [65]:
import statsmodels.api as sm

# training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# add constant to the independent variable
X = sm.add_constant(X)

# perform regession
model = sm.OLS(y, X)
results = model.fit()

#print statistics
print(results.summary())


                              OLS Regression Results                              
Dep. Variable:     Revenue (USD millions)   R-squared:                       0.412
Model:                                OLS   Adj. R-squared:                  0.406
Method:                     Least Squares   F-statistic:                     68.71
Date:                    Tue, 13 Feb 2024   Prob (F-statistic):           6.10e-13
Time:                            14:53:05   Log-Likelihood:                -1267.0
No. Observations:                     100   AIC:                             2538.
Df Residuals:                          98   BIC:                             2543.
Df Model:                               1                                         
Covariance Type:                nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       

Based on the provided data, I can gather the following conclusions:
* R-squared: The R-squared value is 0.412, which means that approximately 41.2% of the variation in ‘Revenue (USD millions)’ can be explained by the ‘Employees’ variable.
* F-statistic: The F-statistic is 68.71, and the associated p-value is very small (6.10e-13), indicating that the ‘Employees’ variable is statistically significant in      predicting ‘Revenue (USD millions)’.
* Coefficients: The coefficient for ‘Employees’ is 0.2384, suggesting that for each additional employee, the ‘Revenue (USD millions)’ increases by approximately 0.2384 units, holding all else constant. The constant term (intercept) is 7.749e+04.
* Omnibus/Prob(Omnibus): The Omnibus test is a test of the skewness and kurtosis of the residual. The Prob(Omnibus) is 0.000, suggesting the residuals are not normally distributed.
* Condition Number: The condition number is large, 3.75e+05. This might indicate that there are strong multicollinearity or other numerical problems.
